# Chapter 5. Object References, Mutability, and Recycling
---

## ToC

1. [Copies Are Shallow by Default](#copies-are-shallow-by-default)

---

## Copies Are Shallow by Default

The easiest way to copy a list (or most built-in mutable collections) is to use the builtin
constructor for the type itself.

**I.** Use Constructor

In [1]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [ ]:
# the copies are equal
l2 == l1

True

In [ ]:
# but refer to two different objects
l2 is l1

False

**II.** Use Shortcut `[:]`

In [1]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = l1[:]
# the copies are equal
l2 == l1

True

In [2]:
# but refer to two different objects
l2 is l1

False

**III.** Use `=`  

In [8]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = l1
l2 == l1

True

In [9]:
l2 is l1

True

For lists and other mutable sequences, the shortcut `l2 = l1[:]` also makes a copy.

However, using the constructor or `[:]` produces a *shallow copy* (i.e., the outermost
container is duplicated, but the copy is filled with references to the same items held
by the original container). This saves memory and causes no problems if all the items
are immutable. But if there are mutable items, this may lead to unpleasant surprises.

**Example:** create a shallow copy of a list containing another list and a tuple,
and then make changes to see how they affect the referenced objects.

In [5]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = l1
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9), 100]


![Figure 90](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/90.PNG)

In [8]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]


![Figure 91](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/91.PNG)

In [10]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 55, 44, 33, 22], (7, 8, 9)]
l2: [3, [66, 55, 44, 33, 22], (7, 8, 9, 10, 11)]


For a mutable object like the list referred by `l2[1]`, the operator `+=` changes the
list in place. This change is visible at `l1[1]`, which is an alias for `l2[1]`.


`+=` on a tuple creates a new tuple and rebinds the variable `l2[2]` here. This is the
same as doing `l2[2] = l2[2] + (10, 11)`. Now the tuples in the last position of
`l1` and `l2` are no longer the same object.

![Figure 92](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/92.PNG)

![Figure 93](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/93.PNG)